In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tfhghbf/train_1.csv


In [80]:
data = pd.read_csv('/kaggle/input/tfhghbf/train_1.csv') #данные

In [81]:
data = data.drop(columns=['smpl', 'id'])
test_sample = data.sample(50000)

In [82]:
SEED = 42 # не меняем

In [83]:
def generate_features_groupby(data, feature_group, func):
    new_features = pd.DataFrame(index=data.index)
    grouped = data.groupby([id, feature_group]).agg([func])
    for col in grouped.index:
        data.join(grouped, on='id', how='left', )
    for col in data.columns:
        for other_col in data.columns:
            if col != other_col:
                new_feature_name = f"{col}_div_{other_col}"
                new_features[new_feature_name] = data[col] / data[other_col]
        print('ok')
        break
    return new_features

In [96]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split

def generate_features_80(data, col1):
    print(col1)
    new_features = pd.DataFrame(index=data.index)
    
    for col in col1:
        for other_col in data.columns:
            if col != other_col:
                new_feature_name = f"{col}_div_{other_col}"
                new_features[new_feature_name] = data[col] / data[other_col]
        print('ok')
    return new_features

def generate_features(data):
    new_features = pd.DataFrame(index=data.index)
    for col in data.columns:
        for other_col in data.columns:
            if col != other_col:
                new_feature_name = f"{col}_div_{other_col}"
                new_features[new_feature_name] = data[col] / data[other_col] #здесь меняем функцию аггрегации
        print('ok')
    return new_features

def train_and_evaluate(data, target, new_features):
    full_data = pd.concat([data, new_features], axis=1)
    train_data, test_data, train_target, test_target = train_test_split(full_data, target,random_state=SEED, test_size=0.2)

    train_pool = Pool(train_data, train_target)
    test_pool = Pool(test_data, test_target)
    
    params = {'loss_function':'Logloss',
          'eval_metric':'AUC', 
          'verbose': 200,
          'early_stopping_rounds': 200,
          'task_type': 'GPU', #Юзайте ГПУ обязательно
          'random_seed': SEED
         }
    import torch
    if not(torch.cuda.is_available()):
        print('юзать гпу')
    cbc_1 = CatBoostClassifier(**params)
    cbc_1.fit(train_data, train_target,
          eval_set=(test_data, test_target), 
          use_best_model=True, 
          plot=True 
         );

    feature_importances = cbc_1.get_feature_importance(prettified=True)
    return feature_importances




In [97]:
import warnings
warnings.filterwarnings('ignore')

In [98]:
best_features = pd.DataFrame(columns=['Feature Id', 'Importances'])

In [ ]:
target = test_sample.target
new_data = test_sample.drop(columns=['target'])

batch_size = 5
for cols in [new_data.columns[i * batch_size: (i+1) * batch_size] for i in range(0, len(new_data.columns), batch_size)]:
    new_features = generate_features_80(new_data, cols)

    feature_importances = train_and_evaluate(new_data, target, new_features)

    top_50_features = feature_importances.nlargest(50, 'Importances')
    important_features = top_50_features[top_50_features['Feature Id'].isin(new_features.columns)]
    best_features = pd.concat([best_features, important_features], ignore_index=True)
    print(new_features)
    print(col)
print("Important Aggregated Features:")
print(important_features)
#в new_data будут топ фичи, только из аггрегированных

Index(['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5'], dtype='object')
ok
ok
ok
ok
ok


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.052041


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.5353040	best: 0.5353040 (0)	total: 172ms	remaining: 2m 52s
200:	test: 0.8126388	best: 0.8153250 (72)	total: 52.6s	remaining: 3m 29s
400:	test: 0.8104049	best: 0.8164441 (256)	total: 1m 44s	remaining: 2m 36s
bestTest = 0.8164440989
bestIteration = 256
Shrink model to first 257 iterations.
        feature_1_div_feature_2  feature_1_div_feature_3  \
131512                 0.517072                -0.538188   
117159                 1.481631                -0.528584   
347002                -0.543978                 1.230126   
322168                 0.855145                 0.973041   
445159                 4.262044                 1.829720   
...                         ...                      ...   
441670                -0.440278                 0.145277   
296722                 0.350726                 0.834538   
84721                  5.161953               -10.486472   
199435                -2.717828                 2.372419   
264805                -0.838984         

...

0:	test: 0.5379534	best: 0.5379534 (0)	total: 172ms	remaining: 2m 51s
200:	test: 0.8028879	best: 0.8106636 (164)	total: 51.6s	remaining: 3m 25s
bestTest = 0.8106636405
bestIteration = 164
Shrink model to first 165 iterations.
        feature_26_div_feature_1  feature_26_div_feature_2  \
131512                  0.931454                  0.481629   
117159                  0.973799                  1.442810   
347002                  1.964124                 -1.068440   
322168                  5.926328                  5.067871   
445159                  0.653569                  2.785540   
...                          ...                       ...   
441670                  8.605199                 -3.788678   
296722                  2.608850                  0.914991   
84721                  -0.154490                 -0.797469   
199435                  0.392316                 -1.066246   
264805                  0.200688                 -0.168374   

        feature_26_div_featur

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.052041


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.5353040	best: 0.5353040 (0)	total: 174ms	remaining: 2m 53s


In [77]:
target = test_sample.target
new_data = test_sample.drop(columns=['target'])
for col in new_data.columns:
    new_features = generate_features_80(new_data, [col])

    feature_importances = train_and_evaluate(new_data, target, new_features)

    top_50_features = feature_importances.nlargest(50, 'Importances')
    important_features = top_50_features[top_50_features['Feature Id'].isin(new_features.columns)]
    best_features = pd.concat([best_features, important_features], ignore_index=True)
    print(new_features)
    print(col)
print("Important Aggregated Features:")
print(important_features)
#в new_data будут топ фичи, только из аггрегированных

ok


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.052041
0:	test: 0.6303988	best: 0.6303988 (0)	total: 74.8ms	remaining: 1m 14s


Default metric period is 5 because AUC is/are not implemented for GPU


200:	test: 0.8011262	best: 0.8020230 (168)	total: 22.9s	remaining: 1m 30s
400:	test: 0.8035278	best: 0.8035399 (392)	total: 45.8s	remaining: 1m 8s
600:	test: 0.8008780	best: 0.8042343 (407)	total: 1m 8s	remaining: 45.5s
bestTest = 0.8042343259
bestIteration = 407
Shrink model to first 408 iterations.
        id_div_feature_1  id_div_feature_2  id_div_feature_3  \
262181     -5.878375e+07     -2.956695e+05     -6.419248e+05   
241055      1.605230e+05      1.116383e+05      7.800751e+04   
342511      1.004327e+06      2.912958e+05      2.008646e+05   
415981      5.784452e+05     -7.648037e+05     -6.332402e+05   
32777       2.289127e+04     -6.791715e+04      2.518624e+04   
...                  ...               ...               ...   
406481     -8.135735e+05     -6.516349e+05     -2.180224e+05   
424565      6.942359e+05      4.680798e+06     -1.136412e+06   
144392     -1.899030e+05     -7.481549e+05     -5.098200e+05   
47073       1.235041e+05     -3.426110e+04      4.661771e+

...

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.052041


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6303988	best: 0.6303988 (0)	total: 74.2ms	remaining: 1m 14s
200:	test: 0.8044661	best: 0.8076988 (185)	total: 23.2s	remaining: 1m 32s


KeyboardInterrupt: 

In [78]:
best_features

,Feature Id,Importances
0,id_div_feature_80,0.900876
1,id_div_feature_201,0.681081
2,id_div_feature_386,0.434020
3,id_div_feature_387,0.433211
4,id_div_feature_240,0.395841
5,id_div_feature_47,0.351153
6,id_div_feature_23,0.338843
7,feature_1_div_feature_80,0.540398
8,feature_1_div_feature_65,0.401562
9,feature_1_div_feature_219,0.388493


In [38]:
new_features.sort_values(by='feature_41_div_id')

,feature_41_div_id,feature_41_div_feature_1,feature_41_div_feature_2,feature_41_div_feature_3,feature_41_div_feature_4,feature_41_div_feature_5,feature_41_div_feature_6,feature_41_div_feature_7,feature_41_div_feature_8,feature_41_div_feature_9,...,feature_41_div_feature_491,feature_41_div_feature_492,feature_41_div_feature_493,feature_41_div_feature_494,feature_41_div_feature_495,feature_41_div_feature_496,feature_41_div_feature_497,feature_41_div_feature_498,feature_41_div_feature_499,feature_41_div_feature_500
28,-0.034995,-3.404064,0.839342,-2.658025,-2.862551,4.544734,1.715835,1.337976,-2.550409,4.137839,...,60.852272,-inf,2.092113,9.144514,4.005559,-2.569294,-0.567128,2.909338,1.440494,0.974741
8,-0.025288,0.208423,0.661813,-0.164010,-0.549666,-1.352404,-0.894427,0.341266,-0.157924,-0.258519,...,0.185498,-inf,0.217356,-0.668775,-0.256811,0.148777,0.122038,0.159206,2.306629,-0.270545
85,-0.018912,-2.993839,-4.207994,-4.636431,-7.890274,-1.387771,-0.940149,1.978893,6.783565,-1.907936,...,1.273910,-1.607562,2.716325,0.619134,-1.879183,3.816994,-1.603730,11.739916,-4.986876,-1.604435
125,-0.010687,4.806028,3.340678,1.719343,2.106516,3.394334,-0.615000,-13.465258,3.190852,14.205464,...,1.606128,-inf,-4.211571,0.637989,13.300608,1.242224,0.789884,-1.622996,-1.566433,103.681367
163,-0.008703,-0.860468,-4.017067,-2.416907,-2.224470,-2.091030,-12.175073,-2.763985,-16.020376,-1.000362,...,1.992991,-1.418530,-4.658281,4.779855,-1.020863,-4.173203,-2.683690,-3.477291,-0.827565,-2.130085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,0.011205,3.730549,37.004239,4.746954,-3.004422,2.813436,2.761781,12.360096,-7.199978,2.368716,...,-8.291779,inf,-1.372499,-2.765198,2.354277,-2.117274,4.024318,22.798961,-5.487508,-14.949236
101,0.022001,1.801757,-16.135525,-29.754039,-1.434651,4.329143,-2.747597,10.747432,-3.379560,1.543823,...,1.370876,2.222066,-10.847453,-1.759889,1.571253,1.335882,-1.318721,-16.936342,-8.285792,2.864434
42,0.028208,1.063329,-1.950645,2.223695,-5.742353,-8.858137,17.403521,5.255961,-4.468826,5.052439,...,-3.635833,inf,3.531416,0.817680,5.117444,-3.366855,0.608251,-3.427315,18.691228,1.569994
12,0.088267,10.894114,0.978487,1.986395,-1.831474,-0.725562,-5.244540,1.839867,0.816409,-2.392210,...,-1.497798,inf,-1.689065,1.245856,-2.291462,2.226662,4.033410,1.548789,-1.535656,-0.968308


In [7]:
! pip install featuretools

In [5]:
data = data.drop(columns=['smpl'])

In [6]:
test_sample = data.sample(50000)

In [7]:
# test_sample['n'] = test_sample.feature_137 / test_sample.feature_121

In [8]:
import featuretools as ft

In [9]:
es = ft.EntitySet(id="id")

In [10]:
es

Entityset: id
  DataFrames:
  Relationships:
    No relationships

In [11]:
es = es.add_dataframe(
    dataframe_name="transactions",
    dataframe=test_sample,
    index="id"
)

In [12]:
es

Entityset: id
  DataFrames:
    transactions [Rows: 50000, Columns: 502]
  Relationships:
    No relationships

In [ ]:
ft.primitives.list_primitives()[ft.primitives.list_primitives().type == 'transform'].values

In [ ]:
data['what']

In [ ]:
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                     target_dataframe_name="transactions",
                                     agg_primitives=["mean", "min"],
                                     trans_primitives=["multiply_numeric"])



/opt/conda/lib/python3.10/site-packages/featuretools/synthesis/deep_feature_synthesis.py:169: UserWarning: Only one dataframe in entityset, changing max_depth to 1 since deeper features cannot be created
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/featuretools/synthesis/dfs.py:321: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['mean', 'min']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible columns for the primitive were found in the data. If the DFS call contained multiple instances of a primitive in the list above, none of them were used.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


In [26]:
# Automatically create new features using DFS
features_matrix, feature_defs = ft.dfs(entityset=es,
                                        target_dataframe_name='transactions',
                                        verbose=True)

print(features_matrix)


/opt/conda/lib/python3.10/site-packages/featuretools/synthesis/deep_feature_synthesis.py:169: UserWarning: Only one dataframe in entityset, changing max_depth to 1 since deeper features cannot be created
  warnings.warn(


Built 501 features
Elapsed: 00:08 | Progress: 100%|██████████
        target  feature_1  feature_2  feature_3  feature_4  feature_5  \
id                                                                      
185073       1  -0.457751  -0.918347  -0.786898   0.286012  -0.941724   
26753        0  -0.848290  -0.240567   0.046197   0.244257   0.673601   
396530       0  -0.303596  -1.994702   0.353005  -0.594653   0.875765   
330429       0   0.142988   0.335448  -1.311498  -1.289505  -0.808531   
51954        0  -1.125883  -0.224397   0.894404  -0.235396  -3.115114   
...        ...        ...        ...        ...        ...        ...   
2472         0   1.031799  -2.396369   0.714633  -0.543259   0.219821   
2891         0   0.524443   0.436333   0.738757  -0.230233  -0.991014   
445340       0  -0.830766   0.653113  -1.161227   0.230022  -0.047900   
242020       0   0.293886   0.705968  -1.252887   0.517696  -0.019847   
310464       0  -0.333560   0.456439   1.209034  -1.022295   1